In [1]:
import pymongo
from pymongo import MongoClient
from dotenv import load_dotenv
import os
import re

In [2]:
load_dotenv()
ATLAS_KEY = os.getenv('ATLAS_KEY')

In [3]:
ATLAS_KEY = os.getenv('ATLAS_KEY')
client = MongoClient(ATLAS_KEY,socketTimeoutMS=5000)
db = client.Db_scraped_IMDB
collection = db.Db_film

#### 1) Quel est le film le plus long ?

In [4]:
def longest_movie(collection) -> str:
        longest_movie = collection.find_one(sort=[("durée", pymongo.DESCENDING)])
        return "Le film le plus long est : ' {} ' avec une durée de {} minutes.".format(longest_movie["titre_original"], longest_movie["durée"])


In [5]:
longest_movie_title = longest_movie(collection)
print(longest_movie_title)

Le film le plus long est : ' Gone with the Wind ' avec une durée de 238 minutes.


#### 2) Quels sont les 5 films les mieux notés ?

In [6]:
def top_film_note(collection)-> str:
    top_rated = collection.find().sort([("score", pymongo.DESCENDING)]).limit(5)
    top_movies = [movie["titre_original"] for movie in top_rated]
    return "Les cinq meilleurs films sont : {}".format (top_movies)

In [7]:
top_film = top_film_note(collection)
print(top_film)

Les cinq meilleurs films sont : ['The Shawshank Redemption', 'The Godfather', 'The Godfather Part II', '12 Angry Men', "Schindler's List"]


#### 3) Dans combien de films a joué Morgan Freeman ? Tom Cruise ?

In [8]:
def count_movies(actor: str) -> str:
        count = collection.count_documents({"acteurs": {"$regex": actor}})
        return "L'acteur ' {} ' a joué dans {} films".format (actor,count)

In [9]:
count_movies_by_actor = count_movies('Morgan Freeman')
print(count_movies_by_actor)

L'acteur ' Morgan Freeman ' a joué dans 4 films


In [10]:
count_movies_by_actor = count_movies('Tom Cruise')
print(count_movies_by_actor)

L'acteur ' Tom Cruise ' a joué dans 0 films


#### 4) Quels sont les 3 meilleurs films d’horreur ? Dramatique ? Comique ?

In [11]:
def top_movies_by_genre(genre: str, n: int) -> str:
        top_movies = collection.find({"genre": {"$regex": genre}}).sort([("score", pymongo.DESCENDING)]).limit(n)
        movie_titles = [movie["titre_original"] for movie in top_movies]
        result = "Les {} meilleurs films de genre ' {} ' sont:\n".format(n, genre)
        for i, title in enumerate(movie_titles):
            result += "{} - {}\n".format(i+1, title)  
        return result

In [12]:
top_movie_by_genre = top_movies_by_genre('Drama', 5)
print(top_movie_by_genre)

Les 5 meilleurs films de genre ' Drama ' sont:
1 - The Shawshank Redemption
2 - The Godfather
3 - The Godfather Part II
4 - 12 Angry Men
5 - Schindler's List



In [13]:
top_movie_by_genre = top_movies_by_genre('Comedy', 5)
print(top_movie_by_genre)

Les 5 meilleurs films de genre ' Comedy ' sont:
1 - Life Is Beautiful
2 - The Intouchables
3 - Modern Times
4 - Back to the Future
5 - City Lights



#### 5) Parmi les 100 films les mieux notés, quel pourcentage sont américains ? Français ?

In [14]:
def percentage_by_country(country: str, n: int) -> str:
        top_rated = list(collection.find().sort([("score", pymongo.DESCENDING)]).limit(n))
        total_count = len(top_rated)
        count_by_country = collection.count_documents({"pays": {"$regex": country}, "_id": {"$in": [movie["_id"] for movie in top_rated]}})
        percentage = count_by_country / total_count * 100
        return f"Parmi les {n} films les mieux notés, {percentage:.2f}% sont de {country}."


In [15]:
percentage_by = percentage_by_country('United States', 100)
print(percentage_by)

Parmi les 100 films les mieux notés, 62.00% sont de United States.


In [16]:
percentage_by_cntry = percentage_by_country('French', 100)
print(percentage_by_cntry)

Parmi les 100 films les mieux notés, 0.00% sont de French.


#### 6) Quel est la durée moyenne d’un film en fonction du genre ?

In [17]:
def average_time_by_genre(collection) -> str:
        result = ""
        genres = collection.distinct("genre")
        for genre in genres:
            time_list = [movie["durée"] for movie in collection.find({"genre": {"$regex": genre}})]
            if time_list:
                average_runtime = sum(time_list) / len(time_list)
                result += f"La durée moyenne d'un film de genre {genre} est de {average_runtime:.2f} minutes.\n"
        return result

In [18]:
average_time_by_genre = average_time_by_genre(collection)
print(average_time_by_genre)

La durée moyenne d'un film de genre Action est de 136.47 minutes.
La durée moyenne d'un film de genre Adventure est de 130.29 minutes.
La durée moyenne d'un film de genre Animation est de 100.96 minutes.
La durée moyenne d'un film de genre Biography est de 144.22 minutes.
La durée moyenne d'un film de genre Comedy est de 105.28 minutes.
La durée moyenne d'un film de genre Crime est de 131.57 minutes.
La durée moyenne d'un film de genre Drama est de 134.75 minutes.
La durée moyenne d'un film de genre Family est de 114.54 minutes.
La durée moyenne d'un film de genre Fantasy est de 123.57 minutes.
La durée moyenne d'un film de genre Film-Noir est de 110.00 minutes.
La durée moyenne d'un film de genre History est de 152.11 minutes.
La durée moyenne d'un film de genre Horror est de 120.60 minutes.
La durée moyenne d'un film de genre Music est de 128.00 minutes.
La durée moyenne d'un film de genre Musical est de 103.00 minutes.
La durée moyenne d'un film de genre Mystery est de 124.77 minute

### * Questions Bonus : 

#### 7) En fonction du genre, afficher la liste des films les plus longs.

In [19]:
def longest_movies_by_genre(genre: str, n: int) -> str:
        top_movies = collection.find({"genre": {"$regex": genre}}).sort([("durée", pymongo.DESCENDING)]).limit(n)
        movie_titles = [movie["titre_original"] for movie in top_movies]
        result = f"Les {n} films les plus longs de genre '{genre}' sont : \n"
        for i, title in enumerate(movie_titles):
            result += f"{i+1}. {title}\n"
        return result

In [20]:
longest_movies_by_genre = longest_movies_by_genre('Drama', 5)
print(longest_movies_by_genre)

Les 5 films les plus longs de genre 'Drama' sont : 
1. Gone with the Wind
2. Once Upon a Time in America
3. Lawrence of Arabia
4. Ben-Hur
5. Seven Samurai



#### 8) En fonction du genre, quel est le coût de tournage d’une minute de film ?

In [21]:
def cost_per_minute_by_genre(genre):
        genre_movies = [movie for movie in collection.find({"genre": genre})]
        total_budget = 0
        total_runtime = 0
        for movie in genre_movies:
            budget_str = movie["budget"]
            if budget_str is None:
                continue
            budget_str = budget_str.split()[0].replace(",", "")
            budget_str = re.sub('[^0-9]', '', budget_str)  # extract only digits
            if not budget_str.isdigit():  # check if the string contains only digits
                continue
            budget = int(budget_str)
            runtime = int(movie["durée"])
            total_budget += budget
            total_runtime += runtime
        cost_per_minute = total_budget / (total_runtime // 60)
        return cost_per_minute


In [22]:
cost_per_minute_by_genre = cost_per_minute_by_genre('Comedy')
print(cost_per_minute_by_genre)

26854296.30666667


#### 9) Quels sont les séries les mieux notés ?